# 🎯 Introducción a LangGraph - Experimentación

Este notebook te introduce a **LangGraph** con ejemplos simples antes de aplicarlo a tu sistema de codificación.

## 📚 ¿Qué es LangGraph?

- **Framework** para construir aplicaciones con LLMs como grafos de estados
- Cada **nodo** = una operación (llamar GPT, procesar datos, etc.)
- Las **aristas** conectan nodos y pueden ser condicionales
- El **estado** fluye entre nodos y se va enriqueciendo

---


## 🔧 Setup Inicial


In [1]:
# Instalar dependencias
!pip install langchain langchain-openai langgraph python-dotenv


  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached ormsgpack-1.12.0-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached orjson-3.11.4-cp312-cp312-win_amd64.whl.metadata (42 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.25.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl (46 kB)
Using cached langgraph_sdk-0.2.9-py3-none-any.whl (56 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 4.9 MB/s  0:00:00
Using cached ti


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Verificar API key
assert os.getenv("OPENAI_API_KEY"), "❌ Falta OPENAI_API_KEY en .env"
print("✅ API Key cargada")


✅ API Key cargada


---

## 🌟 Ejemplo 1: Grafo Simple (Sin LLM)

Empezamos con un grafo que solo procesa números, para entender la mecánica.


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# 1. Definir el ESTADO (datos que fluyen entre nodos)
class EstadoSimple(TypedDict):
    numero: int
    mensaje: str

# 2. Definir NODOS (funciones que procesan el estado)
def nodo_duplicar(state: EstadoSimple) -> EstadoSimple:
    """Duplica el número"""
    nuevo_numero = state["numero"] * 2
    return {
        "numero": nuevo_numero,
        "mensaje": f"Duplicado: {state['numero']} → {nuevo_numero}"
    }

def nodo_sumar_10(state: EstadoSimple) -> EstadoSimple:
    """Suma 10 al número"""
    nuevo_numero = state["numero"] + 10
    return {
        "numero": nuevo_numero,
        "mensaje": state["mensaje"] + f" | Suma: {state['numero']} → {nuevo_numero}"
    }

# 3. Construir el GRAFO
workflow = StateGraph(EstadoSimple)

# Agregar nodos
workflow.add_node("duplicar", nodo_duplicar)
workflow.add_node("sumar", nodo_sumar_10)

# Definir flujo
workflow.set_entry_point("duplicar")  # Empieza aquí
workflow.add_edge("duplicar", "sumar")  # Después va a sumar
workflow.add_edge("sumar", END)  # Y termina

# 4. Compilar
app = workflow.compile()

print("✅ Grafo simple creado")


In [ ]:
# 5. Ejecutar el grafo
resultado = app.invoke({
    "numero": 5,
    "mensaje": "Inicio"
})

print("\n🎯 Resultado:")
print(f"Número final: {resultado['numero']}")
print(f"Trazabilidad: {resultado['mensaje']}")


In [ ]:
# 📊 Visualizar el grafo (si tienes graphviz instalado)
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ No se pudo visualizar: {e}")
    print("\n📝 Ver estructura en formato texto:")
    print(app.get_graph())


---

## 🤖 Ejemplo 2: Grafo con GPT

Ahora agregamos llamadas a OpenAI para analizar texto.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Estado para análisis de texto
class EstadoAnalisis(TypedDict):
    texto_original: str
    idioma: str
    sentimiento: str
    palabras_clave: list
    resumen: str

# Esquemas para respuestas estructuradas
class ResultadoIdioma(BaseModel):
    idioma: str = Field(description="Idioma detectado")
    confianza: float = Field(description="Confianza 0-1")

class ResultadoSentimiento(BaseModel):
    sentimiento: str = Field(description="positivo, negativo o neutral")
    intensidad: float = Field(description="Intensidad 0-1")

# Inicializar LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ LLM inicializado")


In [ ]:
# NODO 1: Detectar idioma
def nodo_detectar_idioma(state: EstadoAnalisis) -> EstadoAnalisis:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Detecta el idioma del texto."),
        ("user", "{texto}")
    ])
    
    chain = prompt | llm.with_structured_output(ResultadoIdioma)
    resultado = chain.invoke({"texto": state["texto_original"]})
    
    print(f"  🌍 Idioma: {resultado.idioma} (confianza: {resultado.confianza})")
    return {**state, "idioma": resultado.idioma}

# NODO 2: Analizar sentimiento
def nodo_analizar_sentimiento(state: EstadoAnalisis) -> EstadoAnalisis:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Analiza el sentimiento del texto."),
        ("user", "{texto}")
    ])
    
    chain = prompt | llm.with_structured_output(ResultadoSentimiento)
    resultado = chain.invoke({"texto": state["texto_original"]})
    
    print(f"  😊 Sentimiento: {resultado.sentimiento} (intensidad: {resultado.intensidad})")
    return {**state, "sentimiento": resultado.sentimiento}

# NODO 3: Extraer palabras clave
def nodo_palabras_clave(state: EstadoAnalisis) -> EstadoAnalisis:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extrae 3-5 palabras clave del texto. Responde solo con las palabras separadas por comas."),
        ("user", "{texto}")
    ])
    
    chain = prompt | llm
    resultado = chain.invoke({"texto": state["texto_original"]})
    
    palabras = [p.strip() for p in resultado.content.split(",")]
    
    print(f"  🔑 Palabras clave: {palabras}")
    return {**state, "palabras_clave": palabras}

# NODO 4: Generar resumen
def nodo_resumen(state: EstadoAnalisis) -> EstadoAnalisis:
    resumen = f"Texto en {state['idioma']} con sentimiento {state['sentimiento']}. Palabras clave: {', '.join(state['palabras_clave'][:3])}"
    print(f"  📝 Resumen: {resumen}")
    return {**state, "resumen": resumen}

print("✅ Nodos con GPT creados")


In [ ]:
# Construir grafo de análisis
workflow_analisis = StateGraph(EstadoAnalisis)

workflow_analisis.add_node("idioma", nodo_detectar_idioma)
workflow_analisis.add_node("sentimiento", nodo_analizar_sentimiento)
workflow_analisis.add_node("palabras", nodo_palabras_clave)
workflow_analisis.add_node("resumen", nodo_resumen)

workflow_analisis.set_entry_point("idioma")
workflow_analisis.add_edge("idioma", "sentimiento")
workflow_analisis.add_edge("sentimiento", "palabras")
workflow_analisis.add_edge("palabras", "resumen")
workflow_analisis.add_edge("resumen", END)

app_analisis = workflow_analisis.compile()

print("✅ Grafo de análisis compilado")


In [ ]:
# Probar con texto de ejemplo
texto_prueba = "Me encanta este producto, tiene muy buen sabor y es versátil para diferentes comidas. Lo recomiendo totalmente."

print(f"\n📄 Analizando: '{texto_prueba}'\n")

resultado = app_analisis.invoke({
    "texto_original": texto_prueba,
    "idioma": "",
    "sentimiento": "",
    "palabras_clave": [],
    "resumen": ""
})

print("\n" + "="*60)
print("📊 RESULTADO FINAL:")
print("="*60)
print(f"Idioma: {resultado['idioma']}")
print(f"Sentimiento: {resultado['sentimiento']}")
print(f"Palabras clave: {resultado['palabras_clave']}")
print(f"Resumen: {resultado['resumen']}")
print("="*60)


---

## 🔀 Ejemplo 3: Transiciones Condicionales

El grafo puede tomar diferentes caminos según el estado. Implementaremos la **Conjetura de Collatz**:

- Si el número es **par**: dividir por 2
- Si es **impar**: multiplicar por 3 y sumar 1
- Repetir hasta llegar a 1


In [ ]:
  
class EstadoCondicional(TypedDict):
    numero: int
    resultado: str
    operaciones: list

def nodo_inicio(state: EstadoCondicional) -> EstadoCondicional:
    print(f"🎬 Empezando con: {state['numero']}")
    return {**state, "operaciones": [f"inicio: {state['numero']}"]}

def nodo_par(state: EstadoCondicional) -> EstadoCondicional:
    nuevo_num = state["numero"] // 2
    print(f"  ➗ {state['numero']} es PAR → dividir por 2 = {nuevo_num}")
    return {
        **state,
        "numero": nuevo_num,
        "operaciones": state["operaciones"] + [f"par: {state['numero']} → {nuevo_num}"]
    }

def nodo_impar(state: EstadoCondicional) -> EstadoCondicional:
    nuevo_num = state["numero"] * 3 + 1
    print(f"  ✖️  {state['numero']} es IMPAR → 3×{state['numero']}+1 = {nuevo_num}")
    return {
        **state,
        "numero": nuevo_num,
        "operaciones": state["operaciones"] + [f"impar: {state['numero']} → {nuevo_num}"]
    }

def nodo_fin(state: EstadoCondicional) -> EstadoCondicional:
    print(f"🎉 ¡Llegó a 1 en {len(state['operaciones'])} pasos!")
    return {
        **state,
        "resultado": f"Llegó a 1 en {len(state['operaciones'])} pasos",
        "operaciones": state["operaciones"] + ["fin"]
    }

# Función que decide el camino
def decidir_camino(state: EstadoCondicional) -> str:
    if state["numero"] == 1:
        return "fin"
    elif state["numero"] % 2 == 0:
        return "par"
    else:
        return "impar"

print("✅ Nodos condicionales creados")


NameError: name 'TypedDict' is not defined

In [ ]:
# Construir grafo con bucles (Conjetura de Collatz)
workflow_cond = StateGraph(EstadoCondicional)

workflow_cond.add_node("inicio", nodo_inicio)
workflow_cond.add_node("par", nodo_par)
workflow_cond.add_node("impar", nodo_impar)
workflow_cond.add_node("fin", nodo_fin)

workflow_cond.set_entry_point("inicio")

# Desde inicio, decidir camino
workflow_cond.add_conditional_edges(
    "inicio",
    decidir_camino,
    {"par": "par", "impar": "impar", "fin": "fin"}
)

# Después de par/impar, volver a decidir (BUCLE)
workflow_cond.add_conditional_edges(
    "par",
    decidir_camino,
    {"par": "par", "impar": "impar", "fin": "fin"}
)

workflow_cond.add_conditional_edges(
    "impar",
    decidir_camino,
    {"par": "par", "impar": "impar", "fin": "fin"}
)

workflow_cond.add_edge("fin", END)

app_cond = workflow_cond.compile()

print("✅ Grafo condicional compilado")


In [ ]:
# Probar con diferentes números
for num in [7, 12, 19]:
    print(f"\n{'='*60}")
    resultado = app_cond.invoke({
        "numero": num,
        "resultado": "",
        "operaciones": []
    })
    print(f"{'='*60}\n")


---

## 🎯 Conceptos Clave Aprendidos

✅ **1. Estado (TypedDict)**: Estructura de datos que fluye entre nodos

✅ **2. Nodos**: Funciones que reciben estado y devuelven estado actualizado

✅ **3. Aristas**: Conexiones entre nodos (`add_edge`)

✅ **4. Aristas Condicionales**: Rutas dinámicas según el estado (`add_conditional_edges`)

✅ **5. Compilación**: Convierte el grafo en una aplicación ejecutable

✅ **6. Bucles**: Un nodo puede volver a sí mismo o a nodos anteriores

---

## ➡️ Siguiente Paso

Continúa con **`02_langgraph_codificacion.ipynb`** para ver cómo aplicar esto a tu sistema de codificación real.
